# Recon2.2plusthreeIndependent.json

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
M = cobra.io.load_json_model('Recon2.2plusthreeIndependent.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [4]:
M.solver='glpk'
sol = M.optimize()
sol.objective_value

0.16888753778858642

##Tyrosine is not enough in the diet (model1)

In [5]:
import cobra
import pandas as pd
import numpy as np

In [6]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeIndependentmap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()
model1=model.copy()

/Users/yanhualiu/Desktop/New_topic/model_building/model_building_exercise_by_yanhua/for different models/Competition between Phe, Tyr, Trp and L-dopa/Making brain specific model and complex model/Modelling for Paper submitted/utils/findBiomarkers.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [7]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model1.reactions.biomass_reaction_INB.flux_expression+
                                              model1.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model1.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model1.objective = quadratic_objective

In [8]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model1.reactions.EX_phe_L_e.lower_bound=-0.1
model1.reactions.EX_phe_L_e.upper_bound=-0.1
model1.reactions.EX_tyr_L_e.lower_bound=-0.02

In [9]:
model1.reactions.EX_dopa_e_INB.upper_bound=1000
model1.reactions.EX_adrnl_e_INB.upper_bound=1000
model1.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model1.reactions.TYRDOPO_INB.upper_bound=0
model1.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model1.reactions.biomass_reaction.lower_bound=0.05
model1.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model1.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model1.reactions.EX_phpyr_e_INB.upper_bound=0
### No PAH in the brain
model1.reactions.PHETHPTOX2_INB.lower_bound=0
model1.reactions.PHETHPTOX2_INB.upper_bound=0
model1.reactions.r0399_INB.lower_bound=0
model1.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model1.reactions.EX_TH_B_INB.upper_bound=1000
model1.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.25513310000000006

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [10]:
model2=model1.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
model2.reactions.PHPYRte.upper_bound=0
model2.reactions.PHPYRte.lower_bound=0
sol = model2.optimize()
sol.objective_value
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

0.17359165779420038

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only Phe deprivation

In [11]:
model5=model2.copy()
model5.reactions.EX_phe_L_e.upper_bound=-0.05
model5.reactions.EX_phe_L_e.lower_bound=-0.05
sol = model5.optimize()
sol.objective_value
fbasol = model5.optimize()
b = show_map(fbasol,map_loc)
b

0.17359165779420038

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only supplying Tyr

In [12]:
model3=model2.copy()
model3.reactions.EX_tyr_L_e.lower_bound=-0.08
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.25405359999999977

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##Even we consider the Phe toxicity for tyrosine hydroxylase, the system uptake more tyrosine could also improve the symptoms of PKU. So, the independent pathway for Phe, Tyr and Trp taking to brain could not describe the PKU disease. 

## Combining the Phe deprivation and Tyr addition

In [13]:
model4=model3.copy()
model4.reactions.EX_phe_L_e.upper_bound=-0.05
model4.reactions.EX_phe_L_e.lower_bound=-0.05
sol = model4.optimize()
sol.objective_value
fbasol = model4.optimize()
b = show_map(fbasol,map_loc)
b

0.2540536

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# For Phe toxicity mechanism

In [28]:
model1.reactions.TH_AToB_INB.upper_bound=0.058
model6=model1.copy()

In [29]:
sol = model6.optimize()
sol.objective_value
fbasol = model6.optimize()
b = show_map(fbasol,map_loc)
b

0.2551331

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [30]:
model7=model6.copy()
model7.genes.get_by_id('HGNC:8582').knock_out()
model7.reactions.PHPYRte.upper_bound=0
model7.reactions.PHPYRte.lower_bound=0
sol = model7.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model7.reactions.Phe_TH_INB.lower_bound=0.055
sol = model7.optimize()
sol.objective_value
fbasol = model7.optimize()
b = show_map(fbasol,map_loc)
b

0.08702650000000002


0.17359165779420038

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only Phe deprivation

In [31]:
model8=model7.copy()
model8.reactions.Phe_TH_INB.lower_bound=0
model8.reactions.EX_phe_L_e.upper_bound=-0.05
model8.reactions.EX_phe_L_e.lower_bound=-0.05
sol = model8.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model8.reactions.Phe_TH_INB.lower_bound=0.055
sol = model8.optimize()
sol.objective_value
fbasol = model8.optimize()
b = show_map(fbasol,map_loc)
b

0.03702650000000001


0.17359165779420035

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Only supplying Tyr

In [32]:
model9=model7.copy()
model9.reactions.Phe_TH_INB.lower_bound=0
model9.reactions.EX_tyr_L_e.lower_bound=-0.08
sol = model9.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model9.reactions.Phe_TH_INB.lower_bound=0.055
sol = model9.optimize()
sol.objective_value
fbasol = model9.optimize()
b = show_map(fbasol,map_loc)
b

0.08702650000000002


0.20100409999999982

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Combining Phe deprivation and Tyr addition

In [33]:
model10=model9.copy()
model10.reactions.Phe_TH_INB.lower_bound=0
model10.reactions.EX_phe_L_e.upper_bound=-0.05
model10.reactions.EX_phe_L_e.lower_bound=-0.05
sol = model10.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model10.reactions.Phe_TH_INB.lower_bound=0.055
sol = model10.optimize()
sol.objective_value
fbasol = model10.optimize()
b = show_map(fbasol,map_loc)
b

0.03702650000000001


0.2540536

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…